In [2]:
#### basic
import pandas as pd
import numpy as np

#### Visulization
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

#### ML
import sklearn
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, TimeSeriesSplit, train_test_split, GroupKFold
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
import lightgbm as lgb
from sklearn import metrics
# import xgboost as xgb

#### Others
import datetime
import os, warnings, random
warnings.filterwarnings('ignore')
import time
import gc

/Users/zhuyuwei/Library/Python/3.7/lib/python/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [10]:
train_X = []
train_Y = []
test_X = []
for i in range(1,6):
    train_path = 'train_FE2_mini_' + str(i) + '.pkl'
    test_path = 'test_FE2_mini_' + str(i) + '.pkl'
    train = pd.read_pickle(train_path)
    train_Y.append(train[['isFraud']])
    train_X.append(train.drop(['isFraud'], axis = 1, inplace = True))
    test_X.append(pd.read_pickle(test_path))

In [3]:
train_X_m = pd.read_csv('DT_M_train.csv',index_col=0,header=None)
test_X_m = pd.read_csv('DT_M_test.csv',index_col=0,header=None)

In [9]:
########################### Vars
#################################################################################
SEED = 42
LOCAL_TEST = False
TARGET = 'isFraud'
START_DATE = datetime.datetime.strptime('2017-11-30', '%Y-%m-%d')

In [ ]:
def make_predictions(train_X, train_Y, test_X, lgb_params, NFOLDS = 5):
    folds = GroupKFold(n_splits=NFOLDS)
 
    split_groups = train_X_m

    tt_df = np.zeros(test_X.shape[0])
    
    oof = np.zeros(len(X))
    for fold_n, (train_index, valid_index) in enumerate(splits):
        start_time = time.time()
        X_train, X_valid = train_X[columns].iloc[train_index], train_X[columns].iloc[valid_index]
        y_train, y_valid = train_Y.iloc[train_index], train_Y.iloc[valid_index]

        dtrain = lgb.Dataset(X_train, label=y_train)
        dvalid = lgb.Dataset(X_valid, label=y_valid)

        # Train
        clf = lgb.train(params, dtrain, 10000, valid_sets = [dtrain, dvalid], verbose_eval=250, early_stopping_rounds=150)

        # Record feature importance
        feature_importances['fold_{}'.format(fold_n + 1)] = clf.feature_importance()

        # Predict and put OOF predicted data into holdout position
        y_pred_valid = clf.predict(X_valid)
        y_pred_valid_vectors[valid_index] = y_pred_valid

        print("Fold {} | AUC: {}".format((fold_n + 1),roc_auc_score(y_valid, y_pred_valid)))
        print('time used {}'.format(time.time()-start_time))

        # Averaging auc score
        score += roc_auc_score(y_valid, y_pred_valid) / 5

        # Averaging predicting value
        y_pred_test_vectors += clf.predict(test_X) / 5

        del X_train, X_valid, y_train, y_valid

        #gabage collector
        gc.collect()

In [10]:
def make_predictions(X, y, test_X, lgb_params, train_X_m,NFOLDS=6):
    
    folds = GroupKFold(n_splits=NFOLDS)

 
    split_groups = train_X_m

    tt_df = np.zeros(test_X.shape[0])   
    oof = np.zeros(len(X))
    
    for fold_, (trn_idx, val_idx) in enumerate(folds.split(X, y, groups=split_groups)):
        print('Fold:',fold_)
        tr_x, tr_y = X.iloc[trn_idx,:], y.iloc[trn_idx]
        vl_x, vl_y = X.iloc[val_idx,:], y.iloc[val_idx]
            
        print(len(tr_x),len(vl_x))
        tr_data = lgb.Dataset(tr_x, label=tr_y)
        vl_data = lgb.Dataset(vl_x, label=vl_y)  

        estimator = lgb.train(
            lgb_params,
            tr_data,
            valid_sets = [tr_data, vl_data],
            verbose_eval = 200,
        )   
        
        pp_p = estimator.predict(test_X)
        tt_df += pp_p/NFOLDS
        
        oof_preds = estimator.predict(vl_x)
        oof[val_idx] = (oof_preds - oof_preds.min())/(oof_preds.max() - oof_preds.min())

        if LOCAL_TEST:
            feature_imp = pd.DataFrame(sorted(zip(estimator.feature_importance(),X.columns)), columns=['Value','Feature'])
            print(feature_imp)
        
        del tr_x, tr_y, vl_x, vl_y, tr_data, vl_data
        gc.collect()
        
    print('OOF AUC:', metrics.roc_auc_score(y, oof))
    if LOCAL_TEST:
        print('Holdout AUC:', metrics.roc_auc_score(tt_df[TARGET], tt_df['prediction']))
    
    return oof,tt_df

In [11]:
########################### Model params
lgb_params = {
            'objective':'binary',
            'boosting_type':'gbdt',
            'metric':'auc',
            'n_jobs':-1,
            'learning_rate':0.01,
            'num_leaves': 491,
            'max_depth':-1,
            'tree_learner':'serial',
            'colsample_bytree': 0.5,
             'subsample_freq':1,
             'subsample':0.7,
             'n_estimators':5000,
              'max_bin':412,
              'verbose':-1,
              'seed': SEED,
              'early_stopping_rounds':100, 
                } 

In [12]:
lgb_params['learning_rate'] = 0.005
lgb_params['n_estimators'] = 10000
lgb_params['early_stopping_rounds'] = 250    
oof, test_predictions = make_predictions(train_X, train_Y, test_X, lgb_params, train_X_m, NFOLDS=6)

Fold: 0
453219 137321
Training until validation scores don't improve for 250 rounds.
[200]	training's auc: 0.971371	valid_1's auc: 0.883661
[400]	training's auc: 0.991777	valid_1's auc: 0.896595
[600]	training's auc: 0.998058	valid_1's auc: 0.903738
[800]	training's auc: 0.999519	valid_1's auc: 0.907277
[1000]	training's auc: 0.999866	valid_1's auc: 0.909738
[1200]	training's auc: 0.999969	valid_1's auc: 0.911713
[1400]	training's auc: 0.999994	valid_1's auc: 0.913595
[1600]	training's auc: 0.999999	valid_1's auc: 0.914867
[1800]	training's auc: 1	valid_1's auc: 0.916259
[2000]	training's auc: 1	valid_1's auc: 0.917384
[2200]	training's auc: 1	valid_1's auc: 0.9184
[2400]	training's auc: 1	valid_1's auc: 0.919006
Early stopping, best iteration is:
[2183]	training's auc: 1	valid_1's auc: 0.918291
Fold: 1
488908 101632
Training until validation scores don't improve for 250 rounds.
[200]	training's auc: 0.971468	valid_1's auc: 0.919061
[400]	training's auc: 0.991612	valid_1's auc: 0.93166

In [13]:
test_predictions

array([0.00016624, 0.00131395, 0.0005106 , ..., 0.00140933, 0.00218048,
       0.00163354])

In [14]:
pred = pd.read_csv('/home/highburyglory95/sample_submission.csv')

pred['isFraud'] = test_predictions
pred.to_csv("submission_fe2_lgb_gcv_.csv", index=False)